<a href="https://colab.research.google.com/github/alotau/colab-notebooks/blob/main/EDH_deck_update_candidates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This project will help reproduce common research I perform whenever a new Magic the Gathering set comes out. Usually I will need to check for new cards that may fit into my existing decks. This involves some special searches using the Scryfall GUI. Now, with this notebook, I can do the same thing in a more repeatable way using the Scryfall API and Python code.

## Fetching cards using Scryfall API

In [1]:
# prompt: write a function to request json data from the scryfall api cards endpoint and provide an example of using it

import requests
import json

def get_scryfall_cards(query=""):
  """Requests JSON data from the Scryfall API cards endpoint.

  Args:
    query: A string representing the search query for the API.

  Returns:
    A JSON object containing the results of the Scryfall API request or an
    empty dictionary if an error occurs.
  """
  url = "https://api.scryfall.com/cards/search"
  params = {"q": query}
  try:
      response = requests.get(url, params=params)
      response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
      return response.json()
  except requests.exceptions.RequestException as e:
      print(f"Error fetching data from Scryfall: {e}")
      return {}



## Displaying card images

The following function will display card images for the cards that are returned from the Scryfall call.

In [2]:
# prompt: using the previously fetched data, write a function to display the card images in a grid that is responsive to the size of the display window

import requests
import json
from IPython.display import display, HTML

def display_cards_grid(data):
    """Displays card images in a responsive grid.

    Args:
      data: The JSON response from the Scryfall API.
    """

    if data and data.get('data'):
      html = '<div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(150px, 1fr)); grid-gap: 10px;">'
      for card in data['data']:
          if 'image_uris' in card and 'normal' in card['image_uris']:
              html += f'<img src="{card["image_uris"]["normal"]}" alt="{card["name"]}" style="width: 100%; height: auto;">'
      html += '</div>'
      display(HTML(html))
    else:
      print("No card data to display or invalid data format.")


## Test it out

If you want, you can try out the function to see how it works. This can be skipped and it won't affect the real work to come later. But if you want to test out displaying the cards, you will need to run this test first.

In [3]:
# Example usage:
# Search for pirate cards with power exactly 4.
cards_data = get_scryfall_cards(query="type:pirate power:4")

# Check if the request was successful and data was retrieved
if cards_data and "data" in cards_data:
    print(f"Found {len(cards_data['data'])} cards.")
    # Access individual cards
    for card in cards_data['data']:
      print(card['name'])
else:
    print("No cards found or an error occurred.")

Found 20 cards.
Adéwalé, Breaker of Chains
Angrath's Marauders
Captivating Crew
Coercive Recruiter
Cutthroat Negotiator
Dire Fleet Ravager
Don Andres, the Renegade
Emberwilde Captain
Fathom Fleet Swordjack
Gorehorn Raider
Marauding Looter
Pirate Ship
Port Razer
Ramirez DePietro
Ramirez DePietro, Pillager
Skyship Buccaneer
Storm Fleet Arsonist
Thunderhead Gunner
Wanted Scoundrels
Zara, Renegade Recruiter


Display the cards that were just fetched. Note that this will only work if you ran the test above.

In [4]:

if cards_data:
    display_cards_grid(cards_data)
else:
    print("Failed to fetch card data.")

# Just Search by Card Name

This little section will let you just search for cards by name. No other parameters. Note that it will search for every card that contains the string you provide to it. So if you search for "Shock" then you'll get every card that has the word "shock" in it somewhere.

In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Assuming the previous code (get_scryfall_data, display_cards_grid) is defined in the same notebook

# Create a text input widget
card_name_input = widgets.Text(
    placeholder='Enter card name',
    description='Card Name:',
    disabled=False
)

# Create an output widget to display results
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()  # Clear previous output
        card_name = card_name_input.value
        if card_name:
            query = f"name=\"{card_name}\""  # Construct query with quotes
            data = get_scryfall_cards(query)
            if data:
                display_cards_grid(data)
            else:
                print(f"No card found with name: {card_name}")
        else:
            print("Please enter a card name.")

# Create a button
search_button = widgets.Button(description="Search")
search_button.on_click(on_button_clicked)


# Display the widgets
display(card_name_input, search_button, output)

Text(value='', description='Card Name:', placeholder='Enter card name')

Button(description='Search', style=ButtonStyle())

Output()

In [6]:
set_name = "fdn" #@param {type:"string"}

query = f"set={set_name} t:pirate colors<=grixis"
cards_data = get_scryfall_cards(query)

if cards_data and "data" in cards_data:
    print(f"Found {len(cards_data['data'])} cards.")
    display_cards_grid(cards_data)
else:
    print("No cards found or an error occurred.")

Found 13 cards.


# Deck Update Checks

Each of the following subsections is a specific check for one of my commander decks to see if there are cards of interest to update those decks.

This is where things get pretty specific using the general functions from above.

## Pirates

[My Grixis pirates deck](https://archidekt.com/decks/1797659/pirates) wants to know if there are any new pirates for it.

In [ ]:
# prompt: using the functions above, create a query that finds all pirates within grixis colors and from a specific set

set_name = "fdn" #@param {type:"string"}

query = f"set={set_name} t:pirate colors<=grixis"
cards_data = get_scryfall_cards(query)

if cards_data and "data" in cards_data:
    print(f"Found {len(cards_data['data'])} cards.")
    display_cards_grid(cards_data)
else:
    print("No cards found or an error occurred.")

Found 13 cards.


## Saskia 7's

[This deck](https://archidekt.com/decks/3030384/saskia_7s) concept taken from Jim LaPage leverages Wild Pair and a pile of creatures with toughness+power=7. Wild Pair really is the secret commander of this deck. Just give a set name and it will return all the creatures with the right sum of power and toughness from that set in Saskia's colors!

In [15]:
# prompt: Using the Scryfall API and the previously created functions above, create an API call for all creatures that have Saskia's colors and whose power and toughness sum to 7 for a particular set.

set_name = "tdc" #@param {type:"string"}

query = f"set={set_name} f:commander type:creature id<=wbrg pt=7 not:reprint"
cards_data = get_scryfall_cards(query)

if cards_data and "data" in cards_data:
    print(f"Found {len(cards_data['data'])} cards.")
    display_cards_grid(cards_data)
else:
    print("No cards found or an error occurred.")

Found 3 cards.


## Bello

I love [my Bello deck](https://archidekt.com/decks/8759830/bigo_bello_enchanto). Look for Red/Green enhantments that cost at least 4 mana.

In [14]:

set_name = "tdm" #@param {type:"string"}

query = f"set={set_name} f:commander type:enchantment id<=rg mv>=4 not:reprint"
cards_data = get_scryfall_cards(query)

if cards_data and "data" in cards_data:
    print(f"Found {len(cards_data['data'])} cards.")
    display_cards_grid(cards_data)
else:
    print("No cards found or an error occurred.")

Found 4 cards.


## Mono-Blue Scrybal

Look for new, blue scry cards for [my scrybal deck](https://archidekt.com/decks/2206475/scrybal).

In [17]:
set_name = "tdm" #@param {type:"string"}

query = f"set={set_name} f:commander id<=u o:\"scry\" not:reprint"
cards_data = get_scryfall_cards(query)

if cards_data and "data" in cards_data:
    print(f"Found {len(cards_data['data'])} cards.")
    display_cards_grid(cards_data)
else:
    print("No cards found or an error occurred.")

Error fetching data from Scryfall: 404 Client Error: Not Found for url: https://api.scryfall.com/cards/search?q=set%3Dtdm+f%3Acommander+id%3C%3Du+o%3A%22scry%22+not%3Areprint
No cards found or an error occurred.


## Mono-White Legends Matter (Dog and Pony Show)

A deck with [Yoshimaru and Keleth](https://archidekt.com/decks/7812709/dog_and_pony_show) at the helm. Look for new Legends to go in this deck.

In [19]:
set_name = "tdc" #@param {type:"string"}

query = f"set={set_name} f:commander id<=w t:Legendary not:reprint"
cards_data = get_scryfall_cards(query)

if cards_data and "data" in cards_data:
    print(f"Found {len(cards_data['data'])} cards.")
    display_cards_grid(cards_data)
else:
    print("No cards found or an error occurred.")

Error fetching data from Scryfall: 404 Client Error: Not Found for url: https://api.scryfall.com/cards/search?q=set%3Dtdc+f%3Acommander+id%3C%3Dw+t%3ALegendary+not%3Areprint
No cards found or an error occurred.
